In [19]:
import uuid
from googleads import adwords

In [48]:
client = adwords.AdWordsClient.LoadFromStorage('/home/tim_su/ai_optimizer/opt/ai_optimizer/googleads.yaml')

In [51]:
CAMPAIGN_ID = 1579088293
ad_group_id = str(56419450781)
PAGE_SIZE = 10

In [53]:
# ID of your customer here
CUSTOMER_SERVICE_ID = '6444395344'

# Load customer account access
client.SetClientCustomerId(CUSTOMER_SERVICE_ID)

In [28]:
# Initialize appropriate service.
ad_group_service = adwords_client.GetService('AdGroupService', version='v201809')

In [54]:
# Initialize appropriate service.
ad_group_ad_service = adwords_client.GetService('AdGroupAdService', version='v201809')

# Construct selector and get all ads for a given ad group.
offset = 0
selector = {
    'fields': ['Id', 'PolicySummary'],
    'predicates': [{
        'field': 'AdGroupId',
        'operator': 'EQUALS',
        'values': [ad_group_id]
    },{
        'field': 'CombinedApprovalStatus',
        'operator': 'EQUALS',
        'values': 'DISAPPROVED'
    }],
  'paging': {
      'startIndex': str(offset),
      'numberResults': str(PAGE_SIZE)
  }
}

more_pages = True
disapproved_count = 0

# Display results.
while more_pages:
    page = ad_group_ad_service.get(selector)
    print(page)
    if 'entries' in page:
        for ad in page['entries']:
            disapproved_count += 1
            policy_summary = ad['policySummary']

            print ('Ad with id "%s" was disapproved with the following policy '
                   'topic entries:' % ad['ad']['id'])
            # Display the policy topic entries related to the ad disapproval.
            for policy_topic_entry in policy_summary['policyTopicEntries']:
                print ('  topic ID: %s, topic name: %s, Help Center URL: %s' % (
                    policy_topic_entry['policyTopicId'],
                    policy_topic_entry['policyTopicName'],
                    policy_topic_entry['policyTopicHelpCenterUrl']))
              # Display the attributes and values that triggered the policy topic.
                policy_topic_evidences = policy_topic_entry['policyTopicEvidences']
                if policy_topic_evidences:
                    for evidence in policy_topic_entry['policyTopicEvidences']:
                        print ('    evidence type: %s'
                             % evidence['policyTopicEvidenceType'])
                        evidence_text_list = evidence['evidenceTextList']
                        if evidence_text_list:
                            for index, evidence_text in enumerate(evidence_text_list):
                                print ('      evidence text[%d]: %s' % (index, evidence_text))

    offset += PAGE_SIZE
    selector['paging']['startIndex'] = str(offset)
    more_pages = offset < int(page['totalNumEntries'])

print( '%d disapproved ads were found.' % disapproved_count)

{
    'totalNumEntries': 0,
    'Page.Type': 'AdGroupAdPage',
    'entries': []
}
0 disapproved ads were found.


In [58]:
ad_group_service = client.GetService('AdGroupService', version='v201809')

# Construct operations and update an ad group.
operations = [{
    'operator': 'SET',
    'operand': {
        'id': ad_group_id,
        'status': 'PAUSED'
    }
}]

# if bid_micro_amount:
#     operations[0]['operand']['biddingStrategyConfiguration'] = {
#         'bids': [{
#             'xsi_type': 'CpcBid',
#             'bid': {
#                 'microAmount': bid_micro_amount,
#             }
#         }]
#     }
ad_groups = ad_group_service.mutate(operations)